代理模式是一种变成模式，他是**将某一个操作转交给另一个对象来实现**

In [31]:
# 简单的代理，就是引入对象，传方法
class A:
    def spam(self, x):
        print ("A： {}".format(x))
    
    def foo(self):
        print ("A: foo")

class B:
    def __init__(self):
        # 引入A的实例
        self._a = A()
    
    def spam(self, x):
        return self._a.spam(x)
    
    def foo(self):
        return self._a.foo()

b = B()
b.spam(1)
b.foo()

A： 1
A: foo


当只有少量方法需要代理的时候，上面那种方法是可取的，当时，如果有大量的方法需要代理，那么使用 `__getattr__()` 方法更加合理：

In [32]:
class B2:
    def __init__(self):
        self._a = A()
        
    # __getattr__ 只有在当前类没有该属性时，才会调用
    def __getattr__(self, name):
        # 取类A中的spam方法
        return getattr(self._a, name)

b = B2()
b.spam(34)

A： 34


或者更高级的点可以实现代理模式，例如：

In [34]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
    
    # 需要实现代理的 __getattr__ ,__setattr__, __delattr__
    def __getattr__(self, name):
        print("getattr:",name)
        return getattr(self._obj, name)
    
    def __setattr__(self, name, value):
        # 如果是私有属性，调用父类的方法
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            # 否则,委托给obj代理
            print('setattr:', name, value)
            setattr(self._obj, name, value)
    
    def __delattr__(self, name):
        if name.startswith('_'):
            super().__delattr__(name)
        else:
            print("delattr:", name)
            delattr(self._obj, name)

当使用这个代理类的时候，需要用它包装其他类：

In [41]:
class Spam:
    def __init__(self, x):
        self.x = x
    
    def bar(self, y):
        print('Spam.bar:',self.x, y)

s = Spam(2)
p = Proxy(s)
# 由于p中没有bar 这个属性，因此会通过 __getattr__ 调用Spam的 bar方法
p.bar(3)

getattr: bar
Spam.bar: 2 3


当使用代理模式是，有些细节需要注意：   

首先，`__getattr__()`实际是一个备用方法，只有在属性不存在时才会进行调用。因此，如果代理类本省有这个属性的话，是不会触发这个方法的。     

其次，`__setattr__()`和`__delattr__()`需要额外的条件来区分代理实例和被代理实例`_obj`属性，通常的约定是**只代理哪些不以`_`开头的属性**，即代理类只暴露被代理类的公共属性。   
  
最后，`__getattr__()`对于**大部分以双下划线开头和结尾的属性（内置属性）不适用**

In [47]:
class ListLike:
    """__getattr__对于双下划线开始和结尾的方法是不能用的，需要一个个去重定义"""

    def __init__(self):
        self._items = []

    def __getattr__(self, name):
        return getattr(self._items, name)

a = ListLike()
ListLike.__dict__


mappingproxy({'__dict__': <attribute '__dict__' of 'ListLike' objects>,
              '__doc__': '__getattr__对于双下划线开始和结尾的方法是不能用的，需要一个个去重定义',
              '__getattr__': <function __main__.ListLike.__getattr__>,
              '__init__': <function __main__.ListLike.__init__>,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'ListLike' objects>})

即便 list 类有 `__len__`方法，但是 `__getattr__`却无法使用代理，因为`__getattr__`钩子对于以双下划线开头的方法无法生效，想要使用，需要在代理类中进行复写

In [49]:
class ListLike:
    """__getattr__对于双下划线开始和结尾的方法是不能用的，需要一个个去重定义"""

    def __init__(self):
        self._items = []

    def __getattr__(self, name):
        return getattr(self._items, name)

    # Added special methods to support certain list operations
    def __len__(self):
        return len(self._items)

    def __getitem__(self, index):
        return self._items[index]

    def __setitem__(self, index, value):
        self._items[index] = value

    def __delitem__(self, index):
        del self._items[index]